In [2]:
# import function from .py file
import pprint, sqlite3
from SPASE_Scraper_Script import SPASE_Scraper
from PathGrabber import getPaths

#print(os.path.abspath(os.getcwd()))
# test path : "/home/jovyan/NASA/NumericalData/DE2" or "/home/jovyan/NASA/NumericalData/ACE/EPAM"
print("Enter root folder you want to search")
folder = input()
print("You entered " + folder)
paths = getPaths(folder)
print("The SPASE records found are:")
print(paths)
#check_Path(os.path.abspath(os.getcwd()))
print("===========================================================================================================")

# print Flag (True prints, False does not)
printFlag = True

for record in paths:
    (RID, RIDField, author, authorField, pub, pubField, pubDate, pubDateField, dataset, datasetField, 
     desc, descField, PI, PIField, AccessRights, licenseField, datalinkField) = SPASE_Scraper(record)
    pubYear = pubDate[0:4]
    author = ", ".join(author)
    if printFlag:
        print("The ResourceID is " + RID + " which was obtained from " + RIDField)
        print("The author(s) are " + author + " which was obtained from " + authorField)
        print("The publication year is " + pubYear + " which was obtained from " + pubDateField)
        print("The publisher is " + pub + " which was obtained from " + pubField)
        print("The dataset is " + dataset + " which was obtained from " + datasetField)
        print("The description is " + desc + " which was obtained from " + descField)
        print("The persistent identifier is " + PI + " which was obtained from " + PIField)
        print("The URLs with their associated product keys obtained from " + datalinkField + """ and their 
              license(s) obtained from """ + licenseField + " are: ")
        pprint.pprint(AccessRights)

    # separate license from the datalink and product keys from AccessRights to store in db
    for k, v in AccessRights.items():
        if not v:
            continue
        else:
            license = k
            datalink = str(v)
            if printFlag:
                print(k + " was assigned to license")
                #for key, val in v.items():
                 #   datalink = key
                print(datalink + " was assigned to datalink")

    # add table to existing database

    def create_tables():
        sql_statements = [ 
            """CREATE TABLE IF NOT EXISTS SPASE_Metadata (
                    id INTEGER PRIMARY KEY, 
                    SPASE_id TEXT NOT NULL UNIQUE, 
                    author TEXT,
                    author_source TEXT,
                    publisher TEXT,
                    publisher_source TEXT,
                    publication_yr TEXT,
                    publication_yr_source TEXT,
                    dataset TEXT,
                    dataset_source TEXT,
                    license TEXT,
                    license_source TEXT,
                    datalink TEXT,
                    datalink_source TEXT,
                    description TEXT,
                    description_source TEXT,
                    PI TEXT,
                    PI_source TEXT
            );"""]

        # create a database connection
        try:
            with sqlite3.connect('SPASE_Data.db') as conn:
                cursor = conn.cursor()
                for statement in sql_statements:
                    cursor.execute(statement)

                conn.commit()
        except sqlite3.Error as e:
            print(e)

    create_tables()

    # insert entries into table

    def add_SPASE(conn, entry):
        sql = '''INSERT INTO SPASE_Metadata(SPASE_id,author,author_source,publisher,publisher_source,
                publication_yr,publication_yr_source,dataset,dataset_source,license,license_source,
                datalink,datalink_source,description,description_source,PI,PI_source)
                VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) '''
        cur = conn.cursor()
        cur.execute(sql, entry)
        conn.commit()
        return cur.lastrowid


    try:
        with sqlite3.connect('SPASE_Data.db') as conn:
            # add a new SPASE Record
            SPASE_Record = (RID, author, authorField, pub, pubField, pubYear, pubDateField, 
                            dataset, datasetField, license, licenseField, datalink, datalinkField,
                            desc, descField, PI, PIField)
            Record_id = add_SPASE(conn, SPASE_Record)
            print(f'Created a SPASE Record with the id {Record_id}')

    except sqlite3.Error as e:
        print(e)
    
    print("===========================================================================================================")

# call .py file directly from notebook
#%run ./SPASE_test.py {path} in notebook
#import sys

#args = sys.argv
#path = args[1] in source file


Enter root folder you want to search


 /home/jovyan/NASA/NumericalData/ACE/EPAM/RATES


You entered /home/jovyan/NASA/NumericalData/ACE/EPAM/RATES
The SPASE records found are:
['/home/jovyan/NASA/NumericalData/ACE/EPAM/RATES/PT5M.xml', '/home/jovyan/NASA/NumericalData/ACE/EPAM/RATES/P1D.xml', '/home/jovyan/NASA/NumericalData/ACE/EPAM/RATES/PT1H.xml', '/home/jovyan/NASA/NumericalData/ACE/EPAM/RATES/PT12S.xml']
Scraping /home/jovyan/NASA/NumericalData/ACE/EPAM/RATES/PT5M.xml
The ResourceID is spase://NASA/NumericalData/ACE/EPAM/RATES/PT5M which was obtained from NumericalData/ResourceID
The author(s) are Gold, R.E., Roelof, E.C., Krimigis, S., Haggerty, D., Armstrong, T.P., Manweiler, J.W., and J.D. Patterson which was obtained from NumericalData/PublicationInfo/Authors
The publication year is 2022 which was obtained from NumericalData/PublicationInfo/PublicationDate
The publisher is NASA Space Physics Data Facility which was obtained from NumericalData/PublicationInfo/PublishedBy
The dataset is ACE Electron Proton Alpha Monitor (EPAM) 5-min Count Rate Data which was obtain

NameError: name 'datalink' is not defined

In [6]:
# add SPASE records without desired links to a text file
textFile = open("../NoLinkRecords.txt", "a")
lines = ["Line 1", "Line 2"]
for line in lines:
    textFile.write(line)
    textFile.write("\n")
textFile.close()

In [7]:
# select row(s) from table
import sqlite3

try:
    with sqlite3.connect("SPASE_Data.db") as conn:
        cur = conn.cursor()
        cur.execute('SELECT * FROM SPASE_Metadata WHERE ')
        rows = cur.fetchall()
        for row in rows:
            print(row)
except sqlite3.Error as e:
    print(e)

(1, 'spase://NASA/DisplayData/ACE/MAG/Daily', 'spase://SMWG/Person/Ruth.Skoug', 'DisplayData/ResourceHeader/Contact/PersonID', 'NASA/GSFC/SPDF', 'DisplayData/AccessInformation/RepositoryID', '', 'DisplayData/PublicationInfo/PublicationDate', 'ACE Daily Survey Plots', 'DisplayData/ResourceHeader/ResourceName', 'Open', 'DisplayData/AccessInformation/AccessRights', "{'https://cdaweb.gsfc.nasa.gov/cgi-bin/gif_walk?plot_type=ace_kp_plots': []}", 'DisplayData/AccessInformation/AccessURL/URL', 'some description', '', 'DisplayData/DOI')


In [3]:
# delete row from table
import sqlite3

try:
    with sqlite3.connect("SPASE_Data.db") as conn:
        cur = conn.cursor()
        delete_stmt = 'DELETE FROM SPASE_Metadata WHERE id = ?'
        cur.execute(delete_stmt, (1,))
        conn.commit()
except sqlite3.Error as e:
    print(e)

In [ ]:
import sqlite3

sql = 'UPDATE SPASE_Metadata SET end_date = ?'

try:
    with sqlite3.connect('SPASE_Data.db') as conn:
        cursor = conn.cursor()
        cursor.execute(sql, ('2015-02-03',) )
        conn.commit()
except sqlite3.Error as e:
    print(e)